## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

### is_row_dominated
* Function to verify if a row2 strategy is dominated by row1 strategy 
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **row1** (int): Integer representing the row we are evaluating for dominating.
    * **row2** (int): Integer representing the row we are evaluating for being dominated.
    * **columns** (set): Set of available columns
    * **weak** (bool): Boolean variable to decide if weak dominated strategy is enabled
* Returns:
    * (bool): True if row1 dominates row2, return False if row1 does not dominate row2

In [3]:
def is_row_dominated(game: List[List[Tuple]], row1: int, row2: int, columns: set, weak: bool) -> bool:
    for col in columns:
        payoff_row1 = game[row1][col][0]
        payoff_row2 = game[row2][col][0]
        if weak:
            if payoff_row1 < payoff_row2:
                return False
        else:
            if payoff_row1 <= payoff_row2:
                return False
    return True

In [4]:
game = [
    [(3, 2), (2, 1)],
    [(1, 2), (0, 3)]
]
columns = {0, 1}
assert type(is_row_dominated(game, 0, 1, columns, weak=False)) == bool
assert is_row_dominated(game, 0, 1, columns, weak=False) == True
assert is_row_dominated(game, 0, 1, columns, weak=True) == True

### is_col_dominated
* Function to verify if a `col2` strategy is dominated by `col1` strategy for Player 2.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **col1** (int): Integer representing the column (strategy for Player 2) we are evaluating for dominating.
    * **col2** (int): Integer representing the column (strategy for Player 2) we are evaluating for being dominated.
    * **rows** (set): Set of available row indices representing Player 1's strategies.
    * **weak** (bool): Boolean variable to decide if weakly dominated strategies are enabled.
* Returns:
    * (bool): True if `col1` dominates `col2`; False if `col1` does not dominate `col2`.


In [5]:
def is_col_dominated(game: List[List[Tuple]], col1: int, col2: int, rows: set, weak: bool) -> bool:
    for row in rows:
        payoff_col1 = game[row][col1][1]
        payoff_col2 = game[row][col2][1]
        if weak:
            if payoff_col1 < payoff_col2:
                return False
        else:
            if payoff_col1 <= payoff_col2:
                return False
    return True

In [6]:
game = [
    [(3, 2), (1, 3)], 
    [(2, 4), (0, 5)]
]
rows = {0, 1}
assert type(is_col_dominated(game, 1, 0, rows, weak=False)) == bool 
assert is_col_dominated(game, 0, 1, rows, weak=False) == False
assert is_col_dominated(game, 0, 1, rows, weak=True) == False


### find_dominated_row
* Function to find a row (strategy for Player 1) that is dominated by another row.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **rows** (set): Set of available row indices representing Player 1's strategies.
    * **columns** (set): Set of available column indices representing Player 2's strategies.
    * **weak** (bool): Boolean variable to decide if weakly dominated strategies are enabled.
* Returns:
    * (int or None): The index of the dominated row (`row2`) if found, otherwise returns `None`.


In [7]:
def find_dominated_row(game: List[List[Tuple]], rows: set, columns: set, weak: bool) -> int:
    for row1 in rows:
        for row2 in rows:
            if row1 != row2 and is_row_dominated(game, row1, row2, columns, weak):
                return row2
    return None

In [8]:
game = [
    [(3, 2), (4, 1)],
    [(1, 2), (0, 3)]
]
rows = {0, 1}
columns = {0, 1}
assert type(find_dominated_row(game, rows, columns, weak=False)) == int
assert find_dominated_row(game, rows, columns, weak=False) == 1
assert find_dominated_row(game, rows, columns, weak=True) == 1


### find_dominated_col
* Function to find a column (strategy for Player 2) that is dominated by another column.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **columns** (set): Set of available column indices representing Player 2's strategies.
    * **rows** (set): Set of available row indices representing Player 1's strategies.
    * **weak** (bool): Boolean variable to decide if weakly dominated strategies are enabled.
* Returns:
    * (int or None): The index of the dominated column (`col2`) if found, otherwise returns `None`.


In [9]:
def find_dominated_col(game: List[List[Tuple]], columns: set, rows: set, weak: bool) -> int:
    for col1 in columns:
        for col2 in columns:
            if col1 != col2 and is_col_dominated(game, col1, col2, rows, weak):
                return col2
    return None

In [10]:
game = [
    [(2, 2), (2, 3)],
    [(1, 3), (0, 3)]
]
columns = {0, 1}
rows = {0, 1}
assert type(find_dominated_col(game, columns, rows, weak=True)) == int
assert find_dominated_col(game, columns, rows, weak=True) == 0
assert find_dominated_col(game, columns, rows, weak=False) == None

### remove_dominated_strategies
* Function to iteratively remove dominated rows (Player 1 strategies) and columns (Player 2 strategies) from the game.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **weak** (bool): Boolean variable to decide if weakly dominated strategies are enabled.
* Returns:
    * **(Tuple[set, set])**: A tuple containing two sets:
        * **rows**: The set of remaining row indices (Player 1's non-dominated strategies).
        * **columns**: The set of remaining column indices (Player 2's non-dominated strategies).


In [11]:
def remove_dominated_strategies(game: List[List[Tuple]], weak: bool) -> Tuple[set, set]:
    rows = set(range(len(game)))
    columns = set(range(len(game[0])))
    while True:
        row_to_remove = find_dominated_row(game, rows, columns, weak)
        if row_to_remove is not None:
            rows.remove(row_to_remove)
            continue
        col_to_remove = find_dominated_col(game, columns, rows, weak)
        if col_to_remove is not None:
            columns.remove(col_to_remove)
            continue
        break
    return rows, columns

In [12]:
game = [
    [(3, 2), (1, 4)],
    [(2, 3), (0, 5)]
]
expected_rows = {0}
expected_columns = {1}
assert type(remove_dominated_strategies(game, weak=False)) == tuple
assert len(remove_dominated_strategies(game, weak=False)) == 2
assert remove_dominated_strategies(game, weak=False) == (expected_rows, expected_columns)



### is_nash_equilibrium
* Function to check if a given strategy pair `(row, col)` is a Nash equilibrium.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **row** (int): The index of the row (Player 1's strategy) to evaluate.
    * **col** (int): The index of the column (Player 2's strategy) to evaluate.
    * **rows** (set): Set of available row indices (Player 1's strategies).
    * **columns** (set): Set of available column indices (Player 2's strategies).
* Returns:
    * **(bool)**: True if the strategy pair `(row, col)` is a Nash equilibrium, False otherwise.


In [13]:
def is_nash_equilibrium(game: List[List[Tuple]], row: int, col: int, rows: set, columns: set) -> bool:
    for other_row in rows:
        if game[other_row][col][0] > game[row][col][0]:
            return False

    for other_col in columns:
        if game[row][other_col][1] > game[row][col][1]:
            return False

    return True

In [14]:
game = [
    [(3, 2), (1, 4)],
    [(2, 1), (0, 5)]
]
rows = {0, 1}
columns = {0, 1}
assert type(is_nash_equilibrium(game, 0, 1, rows, columns)) == bool
assert is_nash_equilibrium(game, 0, 1, rows, columns) == True
assert is_nash_equilibrium(game, 1, 0, rows, columns) == False


### solve_game
* Function to solve a normal-form game by finding all Nash equilibria after removing dominated strategies.
* Args:
    * **game** (list): The normal-form game matrix where each element is a tuple of Player 1's and Player 2's payoffs.
    * **weak** (bool, optional): Boolean variable to decide if weakly dominated strategies are enabled. Defaults to `False`.
* Returns:
    * **(List[Tuple])**: A list of tuples representing the Nash equilibria. Each tuple contains a row and column index `(row, col)` that forms a Nash equilibrium.


In [15]:
def solve_game(game: List[List[Tuple]], weak=False) -> List[Tuple]:
    remaining_rows, remaining_columns = remove_dominated_strategies(game, weak)
    nash_equilibrium = []
    for row in remaining_rows:
        for col in remaining_columns:
            if is_nash_equilibrium(game, row, col, remaining_rows, remaining_columns):
                nash_equilibrium.append((row, col))
    return nash_equilibrium


| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: [(2,1), (0,1),(0,2)]

![alt text](module5part1.png)
![alt text](module5part2.png)

![alt text](module5part3.png)

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3/5 | 2/5 | 1/4 |
|1  | 4/1 | 2/1 | 0/4 |
|2  | 5/2 | 4/2 | 2/3 |

**Solution:**? (strategy indices)

In [16]:
test_game_1 = [
    [(3, 5), (2, 5), (1, 4)],  
    [(4, 1), (2, 1), (0, 4)],  
    [(5, 2), (4, 2), (2, 3)]   
]

solution = solve_game(test_game_1)
solution_weak = solve_game(test_game_1,True)
print(solution)
print(solution_weak)

[(2, 2)]
[(2, 2)]


In [17]:
assert solution == [(2,2)] # insert your solution from above.
assert solution_weak== [(2,2)]

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3/3 | 5/2 | 3/2 |
|1  | 2/5 | 4/1 | 1/3 |
|2  | 1/2 | 0/1 | 2/1 |

**Solution:**? [(0,0)]

In [18]:
test_game_2 = [
    [(3, 3), (5, 2), (3, 2)],
    [(2, 5), (4, 1), (1, 3)],
    [(1, 2), (0, 1), (2, 4)]
    ]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [19]:
print(strong_solution)
print(weak_solution)

[(0, 0)]
[(0, 0)]


In [20]:
assert strong_solution == [(0, 0)]
assert weak_solution == [(0, 0)] # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2/1 | 0/4 | 1/2 |
|1  | 3/0 | 4/1 | 2/3 |
|2  | 1/2 | 2/3 | 3/0 |

**Solution:** None

In [21]:
test_game_3 = [
    [(2, 1), (0, 4), (1, 2)],
    [(3, 0), (4, 1), (2, 3)],  
    [(1, 2), (2, 3), (3, 0)]
]   


strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [22]:
print(strong_solution)
print(weak_solution)

[]
[]


In [23]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solve the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (copy from above)

In [24]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 3)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [25]:
print(strong_solution)
print(weak_solution)

[(0, 1), (0, 2), (2, 1)]
[(0, 1)]


In [26]:
assert strong_solution == [(0, 1), (0, 2), (2, 1)]
assert weak_solution == [(0, 1)] # put solution here